In [1]:
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize
from sklearn.preprocessing import MinMaxScaler #fixed import

In [2]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")
df.head()

,Unnamed: 0,date,title,reason,description,zone,narrative_tfidf,narrative_embeddings,narrative_bert,narrative_tfidf_title,narrative_bert_title,narrative_embeddings_title,tfidf_title
0,0,2021-10-11 12:26:32,CTT - Encomenda entregue danificada e com etiq...,Mau Serviço Prestado,Bom dia venho por este meio apresentar uma rec...,Lisboa,bom dia venho meio apresentar reclamação ctt e...,bom dia venho por este meio apresentar uma rec...,Bom dia venho por este meio apresentar uma rec...,ctt encomenda entregar danificar etiqueta envi...,ctt - encomenda entregue danificada e com etiq...,ctt - encomenda entregue danificada e com etiq...,ctt encomenda entregar danificar etiqueta envi...
1,1,2021-08-31 23:20:15,CTT - Carta com certificado rasgado!,Condições de entrega,- Boa Noite!\r\n\r\n- Venho por este meio recl...,Coimbra,bom noite venho meio reclamar data ás durante ...,- boa noite!\r\n\r\n- venho por este meio recl...,- Boa Noite! - Venho por este meio reclamar qu...,ctt carta certificar rasgar bom noite venho me...,ctt - carta com certificado rasgado! - Boa No...,ctt - carta com certificado rasgado ! - boa no...,ctt carta certificar rasgar
2,2,2021-02-01 19:08:32,CTT - Encomenda registada que nunca chegou ao ...,Atraso de entrega,"Boa tarde,\r\nNo dia 17 de Dezembro de 2020 en...",Aveiro,bom tarde dia dezembro enviar encomenda França...,"boa tarde,\r\nno dia 00 de dezembro de 0000 en...","Boa tarde, No dia 17 de Dezembro de 2020 envie...",ctt encomenda registar nunca chegar destine o ...,ctt - encomenda registada que nunca chegou ao ...,ctt - encomenda registada que nunca chegou ao ...,ctt encomenda registar nunca chegar destine o
3,3,2018-04-19 09:04:34,CTT - Correio Verde,Atraso de entrega,"Bom dia,\n\nPretendia esclarecer uma situação ...",Lisboa,bom dia pretender esclarecer situação ocorrer ...,"bom dia,\n\npretendia esclarecer uma situação ...","Bom dia, Pretendia esclarecer uma situação que...",ctt correio verde bom dia pretender esclarecer...,"ctt - correio verde Bom dia, Pretendia esclar...","ctt - correio verde bom dia,\n\npretendia escl...",ctt correio verde
4,4,2019-07-04 10:20:41,"CTT - Aviso de recepção da carta registada, pr...",Atraso de entrega,venho por este meio solicitar a vossa ajuda pa...,Porto,venho meio solicitar vosso ajudar saber onde e...,venho por este meio solicitar a vossa ajuda pa...,venho por este meio solicitar a vossa ajuda pa...,ctt aviso recepção carta registar preciso urge...,"ctt - aviso de recepção da carta registada, pr...","ctt - aviso de recepção da carta registada , p...",ctt aviso recepção carta registar preciso urge...


In [3]:
#convert reason to numbers
reason_dict ={'Mau Serviço Prestado': 0, 'Condições de entrega': 1, 'Atraso de entrega': 2, 
               'Enganos': 3}
df['reason'].replace(reason_dict, inplace=True)
df['reason'].unique()

array([0, 1, 2, 3], dtype=int64)

In [4]:
y=df['reason']
X = df.loc[:, df.columns != 'reason']

In [5]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# Print the shape of each set
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (15701, 12)
y_train shape: (15701,)
X_val shape: (3364, 12)
y_val shape: (3364,)
X_test shape: (3365, 12)
y_test shape: (3365,)


In [6]:
X_train=X_train['narrative_tfidf']
X_val=X_val['narrative_tfidf']
X_test=X_test['narrative_tfidf']

In [7]:
X_train

9426     bom tarde pensar chegar ponto escrever reclama...
15992    aguardar encomenda Brasil recebi aviso mesmo h...
944      bom tarde fiz u encomenda enviar dia Janeiro s...
3570     constante carteiro entregar carta morada compl...
20225    fiz encomenda alemanho em este encomenda haver...
                               ...                        
2002     bom tarde fiz envio união europeu primeiro vez...
6680     procedir envio encomenda enviar correio regist...
8802     venho meio mostrar descontentamento serviço en...
6405     venho meio reclamar serviço expresso ctt fiz u...
18535    algum motivo todo correspondência enviar trás ...
Name: narrative_tfidf, Length: 15701, dtype: object

In [8]:
X_val

9784     enviar encomenda correio registar Portugal hol...
12557    comprar cd Internet novembro chegar ctt dia de...
15687    após ter receber informativo encomenda haver c...
13491    encomenda origem holando cujo paradeiro em est...
17418    mercadoria chegada mês aguardar resolução núme...
                               ...                        
628      passar dia março efectuei entregar carta vosso...
18173    bom pedir vosso ajudar seguinte esperar encome...
13078    enviar Portugal paço Ferreira encomenda dia de...
19406    venho de este meio fazer reclamação facto rece...
1059     fiz encomenda aliexpress ainda aguardar sempre...
Name: narrative_tfidf, Length: 3364, dtype: object

In [9]:
X_test

1750     fiz encomenda shein chegar Portugal ctt morada...
4563     desde venho mostrar desagrar ctt vez encomenda...
4640     venho meio manifestar descontentamento ctt int...
12810    venho meio reclamar data recebi subsidio paren...
10038    estar esperar encomenda compra efectuar online...
                               ...                        
10820    imenso encomenda internacional chegar Lisboa s...
4812     dia mês outubro chegar Portugal encomenda vind...
4229     venho meio reclamar relativamente processo reg...
893      bom dia gostar fazer reclamação relação encome...
7312     bom tarde seguinte hoje dia março contactei wo...
Name: narrative_tfidf, Length: 3365, dtype: object

# No scalling

In [10]:
def training (clf,X_train, X_val, y_train, y_val,ngram, maxf):
    ngram=ngram #tuplo
    maxf=maxf
    
    # Perform tf-idf vectorization on the training set
    tfidf = TfidfVectorizer(ngram_range=ngram, max_features=maxf)
    X_train_tf = tfidf.fit_transform(X_train)
    
    # Use the same vectorizer to transform the validation set
    X_val_tf = tfidf.transform(X_val)
        
    #Train classifier on the training set
    clf.fit(X_train_tf, y_train)
    
    # Predict the labels on the validation set
    y_val_pred = clf.predict(X_val_tf)
    # Predict the labels on the train set
    y_train_pred = clf.predict(X_train_tf) 

    # Compute the accuracy of the model on the training set
    acc_val = accuracy_score(y_train, y_train_pred)
    print("Training accuracy:", acc_val)
    
    return classification_report(y_val, y_val_pred), confusion_matrix(y_val, y_val_pred, labels=[0,1,2,3])

In [11]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,1), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7036494490796764
              precision    recall  f1-score   support

           0       0.52      0.57      0.55      1195
           1       0.24      0.11      0.15       333
           2       0.67      0.76      0.71      1609
           3       0.25      0.10      0.14       227

    accuracy                           0.58      3364
   macro avg       0.42      0.38      0.39      3364
weighted avg       0.55      0.58      0.56      3364

[[ 687   61  411   36]
 [ 173   35  107   18]
 [ 350   29 1215   15]
 [  99   22   83   23]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.47      0.49      0.48      1195
           1       0.14      0.14      0.14       333
           2       0.64      0.64      0.64      1609
           3       0.15      0.14      0.15       227

    accuracy                           0.50      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.7523724603528438
              precision    recall  f1-score   support

           0       0.57      0.68      0.62      1195
           1       0.39      0.04      0.07       333
           2       0.73      0.83      0.78      1609
           3       0.49      0.19      0.27       227

    accuracy                           0.65      3364
   macro avg       0.55      0.43      0.44      3364
weighted avg       0.62      0.65      0.62      3364

[[ 814   14  349   18]
 [ 211   13   88   21]
 [ 276    0 1328    5]
 [ 129    6   49   43]]


In [12]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(2,2), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.6893828418572066
              precision    recall  f1-score   support

           0       0.55      0.48      0.51      1195
           1       0.27      0.12      0.17       333
           2       0.63      0.83      0.72      1609
           3       0.31      0.08      0.13       227

    accuracy                           0.58      3364
   macro avg       0.44      0.38      0.38      3364
weighted avg       0.55      0.58      0.55      3364

[[ 577   55  543   20]
 [ 143   40  137   13]
 [ 243   27 1330    9]
 [  89   26   93   19]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.48      0.44      0.46      1195
           1       0.17      0.14      0.15       333
           2       0.61      0.72      0.66      1609
           3       0.21      0.14      0.17       227

    accuracy                           0.52      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.959938857397618
              precision    recall  f1-score   support

           0       0.57      0.68      0.62      1195
           1       0.47      0.03      0.05       333
           2       0.72      0.83      0.77      1609
           3       0.43      0.09      0.15       227

    accuracy                           0.65      3364
   macro avg       0.55      0.41      0.40      3364
weighted avg       0.62      0.65      0.60      3364

[[ 814    4  363   14]
 [ 211    9  102   11]
 [ 265    3 1339    2]
 [ 140    3   64   20]]


In [13]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,2), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7007833895930196
              precision    recall  f1-score   support

           0       0.54      0.58      0.56      1195
           1       0.26      0.12      0.16       333
           2       0.68      0.78      0.72      1609
           3       0.29      0.11      0.15       227

    accuracy                           0.59      3364
   macro avg       0.44      0.39      0.40      3364
weighted avg       0.56      0.59      0.57      3364

[[ 689   60  414   32]
 [ 175   39  101   18]
 [ 323   29 1247   10]
 [  99   24   80   24]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.46      0.46      0.46      1195
           1       0.18      0.17      0.18       333
           2       0.63      0.64      0.64      1609
           3       0.14      0.12      0.13       227

    accuracy                           0.50      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.9208330679574549
              precision    recall  f1-score   support

           0       0.58      0.71      0.64      1195
           1       0.47      0.05      0.09       333
           2       0.75      0.83      0.79      1609
           3       0.45      0.17      0.24       227

    accuracy                           0.66      3364
   macro avg       0.56      0.44      0.44      3364
weighted avg       0.64      0.66      0.63      3364

[[ 843   10  323   19]
 [ 207   16   89   21]
 [ 261    3 1339    6]
 [ 139    5   45   38]]


In [14]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(3,3), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.6805299025539775
              precision    recall  f1-score   support

           0       0.52      0.42      0.46      1195
           1       0.18      0.08      0.11       333
           2       0.59      0.81      0.68      1609
           3       0.38      0.07      0.11       227

    accuracy                           0.55      3364
   macro avg       0.41      0.34      0.34      3364
weighted avg       0.51      0.55      0.51      3364

[[ 501   57  620   17]
 [ 135   26  167    5]
 [ 262   41 1303    3]
 [  68   24  120   15]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.48      0.51      0.49      1195
           1       0.18      0.11      0.14       333
           2       0.62      0.67      0.65      1609
           3       0.17      0.11      0.13       227

    accuracy                           0.52      3364
   macro avg       0.3

In [15]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,3), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7026940959174575
              precision    recall  f1-score   support

           0       0.53      0.56      0.54      1195
           1       0.22      0.10      0.13       333
           2       0.67      0.78      0.72      1609
           3       0.23      0.07      0.11       227

    accuracy                           0.59      3364
   macro avg       0.41      0.38      0.38      3364
weighted avg       0.54      0.59      0.56      3364

[[ 666   60  439   30]
 [ 170   32  113   18]
 [ 319   25 1258    7]
 [ 108   26   77   16]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.46      0.43      0.45      1195
           1       0.14      0.15      0.15       333
           2       0.62      0.64      0.63      1609
           3       0.15      0.15      0.15       227

    accuracy                           0.48      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.9638876504681231
              precision    recall  f1-score   support

           0       0.58      0.71      0.64      1195
           1       0.41      0.04      0.07       333
           2       0.75      0.83      0.79      1609
           3       0.48      0.17      0.25       227

    accuracy                           0.66      3364
   macro avg       0.56      0.44      0.44      3364
weighted avg       0.64      0.66      0.63      3364

[[ 852   10  314   19]
 [ 215   12   87   19]
 [ 268    3 1335    3]
 [ 143    4   42   38]]


In [16]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(2,3), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.6860709508948475
              precision    recall  f1-score   support

           0       0.52      0.44      0.48      1195
           1       0.21      0.10      0.13       333
           2       0.61      0.82      0.70      1609
           3       0.25      0.06      0.09       227

    accuracy                           0.56      3364
   macro avg       0.40      0.35      0.35      3364
weighted avg       0.52      0.56      0.52      3364

[[ 528   66  584   17]
 [ 143   33  144   13]
 [ 256   33 1312    8]
 [  82   26  106   13]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.49      0.41      0.44      1195
           1       0.20      0.16      0.18       333
           2       0.60      0.71      0.65      1609
           3       0.19      0.17      0.18       227

    accuracy                           0.51      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.9833131647665754
              precision    recall  f1-score   support

           0       0.58      0.67      0.62      1195
           1       0.53      0.02      0.05       333
           2       0.71      0.86      0.78      1609
           3       0.47      0.06      0.11       227

    accuracy                           0.65      3364
   macro avg       0.57      0.40      0.39      3364
weighted avg       0.63      0.65      0.60      3364

[[ 799    1  387    8]
 [ 208    8  109    8]
 [ 225    3 1381    0]
 [ 141    3   69   14]]


In [17]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,4), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7025030252850137
              precision    recall  f1-score   support

           0       0.51      0.56      0.53      1195
           1       0.21      0.09      0.13       333
           2       0.66      0.76      0.71      1609
           3       0.31      0.09      0.14       227

    accuracy                           0.58      3364
   macro avg       0.42      0.38      0.38      3364
weighted avg       0.54      0.58      0.55      3364

[[ 664   60  446   25]
 [ 177   31  110   15]
 [ 343   35 1224    7]
 [ 111   20   75   21]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.44      0.43      0.44      1195
           1       0.17      0.19      0.18       333
           2       0.61      0.61      0.61      1609
           3       0.16      0.14      0.15       227

    accuracy                           0.47      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.975925100312082
              precision    recall  f1-score   support

           0       0.57      0.72      0.64      1195
           1       0.44      0.04      0.07       333
           2       0.76      0.82      0.79      1609
           3       0.48      0.16      0.24       227

    accuracy                           0.66      3364
   macro avg       0.56      0.44      0.43      3364
weighted avg       0.64      0.66      0.63      3364

[[ 866    8  301   20]
 [ 218   12   86   17]
 [ 287    3 1316    3]
 [ 146    4   40   37]]


In [18]:
def nr_features(t):
    # create a TF-IDF vectorizer with desired configurations
    vectorizer = TfidfVectorizer(ngram_range=t)
    # fit and transform the text data
    tfidf_matrix = vectorizer.fit_transform(X_train)
    # get the number of features
    num_features = len(vectorizer.get_feature_names_out())
    print("Number of features when n-gram is {ng}: {nf}".format(ng=t,nf=num_features))

In [19]:
nr_features((1,1))
nr_features((1,2))
nr_features((2,2))
nr_features((1,3))
nr_features((3,3))
nr_features((2,3))
nr_features((1,4))

Number of features when n-gram is (1, 1): 21021
Number of features when n-gram is (1, 2): 388817
Number of features when n-gram is (2, 2): 367796
Number of features when n-gram is (1, 3): 1212232
Number of features when n-gram is (3, 3): 823415
Number of features when n-gram is (2, 3): 1191211
Number of features when n-gram is (1, 4): 2193025


The max_df parameter in the TF-IDF vectorizer specifies the threshold value for document frequency, beyond which terms will be ignored. This parameter can be used to remove words that are too common and may not be informative for text classification. However, setting max_df too low can also result in the removal of important words.

Choosing an appropriate max_df value can be somewhat subjective and may depend on the specific text dataset and the classification task. A common approach is to start with a value of 0.95 and gradually reduce it until the desired number of features is obtained. If the corpus is small, choose a smaller max_df value (e.g., 0.5 or lower) to avoid overfitting.
If the corpus is large, choose a higher max_df value (e.g., 0.9 or higher) to remove common words and reduce dimensionality.
If the text dataset contains many rare words or is noisy, it may be necessary to set max_df to a lower value to remove those words.

In [20]:
import pandas as pd
import numpy as np
from collections import Counter

# Get the number of unique words in the complaints
unique_words = set(" ".join(df['narrative_tfidf']).split())
print("Number of unique words:", len(unique_words))

# Get the frequency of the words in the complaints
word_freq = Counter(" ".join(df['narrative_tfidf']).split())
# print("Frequency of words:", word_freq)

# Get the length of each complaint
complaint_lengths = [len(x.split()) for x in df['narrative_tfidf']]
# print("Length of complaints:", complaint_lengths)


Number of unique words: 25140


# with scaling

In [21]:
def training2(clf,X_train, X_val, y_train, y_val,ngram, maxf):
    ngram=ngram #tuplo
    maxf=maxf
    
    # Perform tf-idf vectorization on the training set
    tfidf = TfidfVectorizer(ngram_range=ngram, max_features=maxf)
    X_train_tf = tfidf.fit_transform(X_train)
    
    # Use the same vectorizer to transform the validation set
    X_val_tf = tfidf.transform(X_val)
    
    from sklearn.preprocessing import MaxAbsScaler

   # ValueError: Negative values in data passed to MultinomialNB (input X) if standard scaler is used!!!
    # use to_array if sparse matrix
    # no memory available for to_array()
    scaler = MaxAbsScaler()
    X_train_scaled = scaler.fit_transform(X_train_tf)
    X_val_scaled= scaler.transform(X_val_tf)

    #Train classifier on the training set
    clf.fit(X_train_scaled, y_train)
    
    # Predict the labels on the validation set
    y_val_pred = clf.predict(X_val_scaled)
    # Predict the labels on the train set
    y_train_pred = clf.predict(X_train_scaled) 

    # Compute the accuracy of the model on the training set
    acc_val = accuracy_score(y_train, y_train_pred)
    print("Training accuracy:", acc_val)
    
    return classification_report(y_val, y_val_pred), confusion_matrix(y_val, y_val_pred, labels=[0,1,2,3])

In [22]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,1), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7036494490796764
              precision    recall  f1-score   support

           0       0.52      0.57      0.55      1195
           1       0.24      0.11      0.15       333
           2       0.67      0.76      0.71      1609
           3       0.25      0.10      0.14       227

    accuracy                           0.58      3364
   macro avg       0.42      0.38      0.39      3364
weighted avg       0.55      0.58      0.56      3364

[[ 687   61  411   36]
 [ 173   35  107   18]
 [ 350   29 1215   15]
 [  99   22   83   23]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.46      0.49      0.47      1195
           1       0.17      0.15      0.16       333
           2       0.64      0.64      0.64      1609
           3       0.16      0.14      0.15       227

    accuracy                           0.51      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.7523724603528438
              precision    recall  f1-score   support

           0       0.57      0.68      0.62      1195
           1       0.39      0.04      0.07       333
           2       0.73      0.83      0.78      1609
           3       0.49      0.19      0.27       227

    accuracy                           0.65      3364
   macro avg       0.55      0.43      0.44      3364
weighted avg       0.62      0.65      0.62      3364

[[ 814   14  349   18]
 [ 211   13   88   21]
 [ 276    0 1328    5]
 [ 129    6   49   43]]


In [23]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(2,2), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.6893828418572066
              precision    recall  f1-score   support

           0       0.55      0.48      0.51      1195
           1       0.27      0.12      0.17       333
           2       0.63      0.83      0.72      1609
           3       0.31      0.08      0.13       227

    accuracy                           0.58      3364
   macro avg       0.44      0.38      0.38      3364
weighted avg       0.55      0.58      0.55      3364

[[ 577   55  543   20]
 [ 143   40  137   13]
 [ 243   27 1330    9]
 [  89   26   93   19]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.49      0.43      0.46      1195
           1       0.18      0.15      0.17       333
           2       0.61      0.72      0.66      1609
           3       0.22      0.15      0.17       227

    accuracy                           0.52      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.959938857397618
              precision    recall  f1-score   support

           0       0.57      0.68      0.62      1195
           1       0.47      0.03      0.05       333
           2       0.72      0.83      0.77      1609
           3       0.43      0.09      0.15       227

    accuracy                           0.65      3364
   macro avg       0.55      0.41      0.40      3364
weighted avg       0.62      0.65      0.60      3364

[[ 814    4  363   14]
 [ 211    9  102   11]
 [ 265    3 1339    2]
 [ 140    3   64   20]]


In [15]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training2(classifier,X_train, X_val, y_train, y_val,(1,2), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.4971657856187504
              precision    recall  f1-score   support

           0       0.38      0.01      0.02      1195
           1       0.44      0.01      0.02       333
           2       0.48      0.99      0.65      1609
           3       1.00      0.01      0.03       227

    accuracy                           0.48      3364
   macro avg       0.57      0.26      0.18      3364
weighted avg       0.47      0.48      0.32      3364

[[  12    0 1183    0]
 [   3    4  326    0]
 [  14    2 1593    0]
 [   3    3  218    3]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.46      0.45      0.46      1195
           1       0.16      0.17      0.16       333
           2       0.64      0.65      0.64      1609
           3       0.19      0.16      0.17       227

    accuracy                           0.50      3364
   macro avg       0.3

In [24]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(3,3), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.6805299025539775
              precision    recall  f1-score   support

           0       0.52      0.42      0.46      1195
           1       0.18      0.08      0.11       333
           2       0.59      0.81      0.68      1609
           3       0.38      0.07      0.11       227

    accuracy                           0.55      3364
   macro avg       0.41      0.34      0.34      3364
weighted avg       0.51      0.55      0.51      3364

[[ 501   57  620   17]
 [ 135   26  167    5]
 [ 262   41 1303    3]
 [  68   24  120   15]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.48      0.51      0.49      1195
           1       0.17      0.10      0.13       333
           2       0.62      0.67      0.64      1609
           3       0.17      0.12      0.14       227

    accuracy                           0.52      3364
   macro avg       0.3

In [25]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,3), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7026940959174575
              precision    recall  f1-score   support

           0       0.53      0.56      0.54      1195
           1       0.22      0.10      0.13       333
           2       0.67      0.78      0.72      1609
           3       0.23      0.07      0.11       227

    accuracy                           0.59      3364
   macro avg       0.41      0.38      0.38      3364
weighted avg       0.54      0.59      0.56      3364

[[ 666   60  439   30]
 [ 170   32  113   18]
 [ 319   25 1258    7]
 [ 108   26   77   16]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.46      0.43      0.44      1195
           1       0.15      0.16      0.16       333
           2       0.62      0.65      0.64      1609
           3       0.18      0.16      0.17       227

    accuracy                           0.49      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.9638876504681231
              precision    recall  f1-score   support

           0       0.58      0.71      0.64      1195
           1       0.41      0.04      0.07       333
           2       0.75      0.83      0.79      1609
           3       0.48      0.17      0.25       227

    accuracy                           0.66      3364
   macro avg       0.56      0.44      0.44      3364
weighted avg       0.64      0.66      0.63      3364

[[ 852   10  314   19]
 [ 215   12   87   19]
 [ 268    3 1335    3]
 [ 143    4   42   38]]


In [26]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(2,3), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.6860709508948475
              precision    recall  f1-score   support

           0       0.52      0.44      0.48      1195
           1       0.21      0.10      0.13       333
           2       0.61      0.82      0.70      1609
           3       0.25      0.06      0.09       227

    accuracy                           0.56      3364
   macro avg       0.40      0.35      0.35      3364
weighted avg       0.52      0.56      0.52      3364

[[ 528   66  584   17]
 [ 143   33  144   13]
 [ 256   33 1312    8]
 [  82   26  106   13]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.50      0.40      0.44      1195
           1       0.18      0.16      0.17       333
           2       0.60      0.72      0.66      1609
           3       0.19      0.15      0.17       227

    accuracy                           0.51      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.9833131647665754
              precision    recall  f1-score   support

           0       0.58      0.67      0.62      1195
           1       0.53      0.02      0.05       333
           2       0.71      0.86      0.78      1609
           3       0.47      0.06      0.11       227

    accuracy                           0.65      3364
   macro avg       0.57      0.40      0.39      3364
weighted avg       0.63      0.65      0.60      3364

[[ 799    1  387    8]
 [ 208    8  109    8]
 [ 225    3 1381    0]
 [ 141    3   69   14]]


In [27]:
# cf: The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
for classifier in [KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
                   XGBClassifier(), MultinomialNB(),SVC(kernel='linear')]:
    print(classifier)
    cm,cf=training(classifier,X_train, X_val, y_train, y_val,(1,4), maxf=None)
    print(cm)
    print(cf)

KNeighborsClassifier()
Training accuracy: 0.7025030252850137
              precision    recall  f1-score   support

           0       0.51      0.56      0.53      1195
           1       0.21      0.09      0.13       333
           2       0.66      0.76      0.71      1609
           3       0.31      0.09      0.14       227

    accuracy                           0.58      3364
   macro avg       0.42      0.38      0.38      3364
weighted avg       0.54      0.58      0.55      3364

[[ 664   60  446   25]
 [ 177   31  110   15]
 [ 343   35 1224    7]
 [ 111   20   75   21]]
DecisionTreeClassifier()
Training accuracy: 0.9993630978918541
              precision    recall  f1-score   support

           0       0.43      0.42      0.42      1195
           1       0.18      0.21      0.19       333
           2       0.60      0.61      0.60      1609
           3       0.18      0.16      0.17       227

    accuracy                           0.47      3364
   macro avg       0.3

d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training accuracy: 0.975925100312082
              precision    recall  f1-score   support

           0       0.57      0.72      0.64      1195
           1       0.44      0.04      0.07       333
           2       0.76      0.82      0.79      1609
           3       0.48      0.16      0.24       227

    accuracy                           0.66      3364
   macro avg       0.56      0.44      0.43      3364
weighted avg       0.64      0.66      0.63      3364

[[ 866    8  301   20]
 [ 218   12   86   17]
 [ 287    3 1316    3]
 [ 146    4   40   37]]


In [28]:
nr_features((1,1))
nr_features((1,2))
nr_features((2,2))
nr_features((1,3))
nr_features((3,3))
nr_features((2,3))
nr_features((1,4))

Number of features when n-gram is (1, 1): 21021
Number of features when n-gram is (1, 2): 388817
Number of features when n-gram is (2, 2): 367796
Number of features when n-gram is (1, 3): 1212232
Number of features when n-gram is (3, 3): 823415
Number of features when n-gram is (2, 3): 1191211
Number of features when n-gram is (1, 4): 2193025


"It's generally a good idea to scale your data after feature engineering, because feature engineering can significantly alter the distribution of your data and make it difficult to determine the appropriate scaling method. Scaling before feature engineering can also lead to the loss of important information, especially if you are using techniques that are sensitive to the distribution of the data, such as PCA or kernel methods.

By performing feature engineering first, you can gain a better understanding of the distribution of your data and determine the appropriate scaling method based on the transformed features. For example, if you use a logarithmic transformation to make a feature more normally distributed, you may need to use a scaling method that is appropriate for normally distributed data, such as StandardScaler. On the other hand, if you use a quantile transformation to make a feature more uniformly distributed, you may need to use a scaling method that is appropriate for uniformly distributed data, such as MinMaxScaler.

It's also important to note that feature engineering can create new features with different scales and ranges, which can affect the performance of models that rely on distance-based calculations, such as KNN. In these cases, it's important to ensure that all features have the same scale and range, either by scaling each feature individually or by using a scaling method that takes into account the range of all features."